In [3]:
import pandas as pd
import torch

# 数据清洗注意事项

## 关于分词
- 类别标注是按字符标注的，需要手动按字符分词
- 注意分词后会在前后加入`[CLS]`和`[SEP]`，正常应该比标签长2

以下是示例

In [4]:
train_fullname = './data/train_data_public.csv'
train_raw = pd.read_csv(train_fullname)
bert_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-chinese')

Using cache found in C:\Users\Justi/.cache\torch\hub\huggingface_pytorch-transformers_main


**自动分词**

In [5]:
output = bert_tokenizer(train_raw.iloc[187,1])
print(bert_tokenizer.decode(output['input_ids']))
print(train_raw.iloc[187,2])
print(f'得到的分词数：{len(output["input_ids"])}, 标注数：{len(train_raw.iloc[187,2].split())}')

[CLS] 卡 多 不 好 批?? 。 我 15. 8 额 度 ， 申 了 两 次 没 有 批 ， 就 因 为 多 头 授 信 。 [SEP]
B-PRODUCT O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
得到的分词数：33, 标注数：32


**按字符分词**

In [6]:
output = bert_tokenizer(list(train_raw.iloc[187,1]), is_split_into_words=True)
print(bert_tokenizer.decode(output['input_ids']))
print(train_raw.iloc[187,2])
print(f'得到的分词数：{len(output["input_ids"])}, 标注数：{len(train_raw.iloc[187,2].split())}')

[CLS] 卡 多 不 好 批?? 。 我 1 5. 8 额 度 ， 申 了 两 次 没 有 批 ， 就 因 为 多 头 授 信 。 [SEP]
B-PRODUCT O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
得到的分词数：34, 标注数：32


## 关于空格
- **似乎只有此现象影响了源数据字符串和标记串的长度一致性**
- **似乎没有观察到字符串比标记串长的情况**
<br>

- 文本的空格同样被标上了标记
- 文本的换行符没有标记
- 使用`strip()`去除换行符和空格时，会丢弃字符串前后的空格
- 处理后的字符串可能会比标记串短
- 根据观察，这些空格都出现在行尾，可直接截断标记串

In [25]:
for idx,row in train_raw.iterrows():
    text, label = row['text'], row['BIO_anno']
    if len(list(text.strip())) != len(label.strip().split()):
        print(f'id:\t\t{idx}\ntext:\tbegin|{text}|end\nsplit:\t{"|".join(list(text.strip()))}\nlabel:\t{"|".join(label.strip().split())}\ntext len:\t{len(list(text.strip()))}\nlabel len\t{len(label.strip().split())}')
        print('---------------------------------------')

id:		933
text:	begin|楼主，解决了吗？我的跟你一样，好操心 |end
split:	楼|主|，|解|决|了|吗|？|我|的|跟|你|一|样|，|好|操|心
label:	O|O|O|O|O|O|O|O|O|O|O|O|O|O|O|O|B-COMMENTS_ADJ|I-COMMENTS_ADJ|O
text len:	18
label len	19
---------------------------------------
id:		1859
text:	begin|广发_178k　|end
split:	广|发|_|1|7|8|k
label:	B-BANK|I-BANK|O|O|O|O|O|O
text len:	7
label len	8
---------------------------------------
id:		2283
text:	begin|中行_7.4k　|end
split:	中|行|_|7|.|4|k
label:	B-BANK|I-BANK|O|O|O|O|O|O
text len:	7
label len	8
---------------------------------------
id:		2300
text:	begin|招行_经典白100k　|end
split:	招|行|_|经|典|白|1|0|0|k
label:	B-BANK|I-BANK|O|B-PRODUCT|I-PRODUCT|I-PRODUCT|O|O|O|O|O
text len:	10
label len	11
---------------------------------------
id:		3834
text:	begin|工行无界白_100k　|end
split:	工|行|无|界|白|_|1|0|0|k
label:	B-BANK|I-BANK|B-PRODUCT|I-PRODUCT|I-PRODUCT|O|O|O|O|O|O
text len:	10
label len	11
---------------------------------------
id:		5721
text:	begin|中信好久不能调额了　　　　　　　　|end
split:	中|信|好|久|不|能|调|额|了
label:	B-BAN